# Chapter 3: Linear Methods for Regression

We have an input vector $X^T = (X_1, X_2, ..., X_p)$ and want to predict a real-valued ouput $Y$.  The linear regression model has the form

$$ f(X) = \beta_0 + \sum_{j=1}^p \, X_j \, \beta_j $$

where the $\beta_j$ are unknown coefficients.  Note that if we add an element $X_0=1$ to $X^T = (1, X_1, X_2, ..., X_p)$ and absorb the *intercept* coefficient $\beta_0$ into the summation, we can write the regression model as a simple inner product

$$ f(X) = \sum_{j=0}^p \, X_j \, \beta_j $$

  Typically we have a set of training data $(x_1, y_1) ... (x_N, y_N)$ from which to estimate these coefficients.  Each $x_i = (1, x_{i1}, x_{i2}, ...., x_{ip})^T$ is a vector of feature measurements for the $i$th case.  The most popular estimation method is *least squares*, in which we pick coefficients $\beta = (\beta_0, \beta_1, ..., \beta_p)^T$ to minimize the residual sum of squares

$$ 
\begin{aligned}
{\rm RSS}(\beta) &= \sum_{i=1}^{N} \left[y_i - f(x_i)\right]^2 \\
&= \sum_{i=1}^{N} \left[ y_i - \sum_{j=1}^p \, x_{ij} \, \beta_j \right]^2
\end{aligned}
$$

If we construct an $N \times (p+1)$ matrix from the vectors $x_i$ (sometimes called the design matrix) we can express $RSS(\beta)$ using only matrix operations

$$
{\bf X} = 
\begin{bmatrix}
    1       & x_{11} & x_{12} & \dots & x_{1p} \\
    1       & x_{21} & x_{22} & \dots & x_{2p} \\
    \vdots  & \vdots & \vdots & \ddots & \vdots \\
    1       & x_{N1} & x_{N2} & \dots & x_{Np}
\end{bmatrix}
\quad
{\bf y} = 
\begin{bmatrix}
    y_{1} \\
    y_{2} \\
    \vdots \\
    y_{N}
\end{bmatrix}
\quad
\beta = 
\begin{bmatrix}
    \beta_{0} \\
    \beta_{1} \\
    \beta_{2} \\
    \vdots \\
    \beta_{p}
\end{bmatrix}
$$

$$ {\rm RSS}(\beta) = ({\bf y} - {\bf X} \beta)^T ({\bf y} - {\bf X} \beta) $$

This is a quadratic function in the $p+1$ coefficients $\beta$. Differentiating with respect to $\beta$ we obtain

$$
\begin{aligned}
\frac{\partial {\rm RSS}}{\partial \beta} &= -2 {\bf X}^T ({\bf y} - {\bf X} \beta) \\
\frac{\partial^2 {\rm RSS}}{\partial \beta \, \partial \beta^T} &= 2 {\bf X}^T {\bf X}
\end{aligned}
$$

Assuming (for the moment) that ${\bf X}$ has full column rank (i.e. the columns=features are linearly independent), and hence ${\bf X}^T {\bf X}$ is positive definite, we set the first derivative to zero

$$ {\bf X}^T ({\bf y} - {\bf X} \beta) = 0 $$

to obtain the unique solution

$$ \hat{\beta} = ({\bf X}^T {\bf X})^{-1} {\bf X}^T {\bf y} $$

# Prostate Data Examples

Lets explore the prostate data set provided at the book website https://statweb.stanford.edu/~tibs/ElemStatLearn/data.html

In [11]:
import pandas
import numpy as np

In [159]:
df = pandas.read_csv('../data/prostate.data', delimiter='\t', index_col=0)

In [4]:
df.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [218]:
predictors = ['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']
target = ['lpsa']
X = df.loc[df['train']=='T', predictors].values
y = df.loc[df['train']=='T', target].values

N, p = X.shape
print('N=', N, 'p=', p)

X = np.matrix(X)
y = np.matrix(y)

X = (X - X.mean(axis=0)) / X.std(axis=0)
X = np.c_[np.ones(X.shape[0]), X]

beta = np.linalg.inv(X.T * X) * X.T * y
yhat = X * beta

# estimate the variance of y
sig2_hat = float((y-yhat).T*(y-yhat)/(N-p-1))

# calculate the covariance matrix of beta
var_beta = np.linalg.inv(X.T * X) * sig2_hat

print('sig2_hat=',sig2_hat)
print('shape(beta)=', var_beta.shape)



N= 67 p= 8
sig2_hat= 0.5073514562053173
shape(beta)= (9, 9)


In [213]:
sig2_hat


matrix([[ 0.50735146]])

In [183]:
np.linalg.inv(X.T * X) * X.T * y

matrix([[ 0.42917013],
        [ 0.57654319],
        [ 0.61402   ],
        [-0.01900102],
        [ 0.14484808],
        [ 0.73720864],
        [-0.20632423],
        [-0.02950288],
        [ 0.00946516]])

In [126]:
X = df[predictors].loc[df['train']=='T']
df_Xtrain.corr()
df_Ytrain = df[target].loc[df['train']=='T']

In [155]:
Xtrain_mean = df_Xtrain.mean()
Xtrain_std = df_Xtrain.std()
X = ((df_Xtrain - df_Xtrain.mean()) / df_Xtrain.std()).values
y = df_Ytrain.values

In [156]:
X = np.c_[np.ones(X.shape[0]), X]

In [157]:
X = np.matrix(X)
y = np.matrix(y).T

In [158]:
np.linalg.pinv(X.T * X) * X.T * y

matrix([[ 2.45234509],
        [ 0.71640701],
        [ 0.2926424 ],
        [-0.14254963],
        [ 0.2120076 ],
        [ 0.30961953],
        [-0.28900562],
        [-0.02091352],
        [ 0.27734595]])

In [82]:
np.matrix(y).T.shape

(67, 1)

In [30]:
Xp = (X - X.mean(axis=0)) / X.std(axis=0) * np.sqrt(96)
print(Xp.mean(axis=0))
print(Xp.std(axis=0))
print(Xp.var(axis=0))

[  1.83129571e-16  -3.25054989e-15   4.12041535e-15  -6.95892370e-16
   5.49388713e-16   3.29633228e-16   2.42646682e-16   1.78551332e-16]
[ 9.79795897  9.79795897  9.79795897  9.79795897  9.79795897  9.79795897
  9.79795897  9.79795897]
[ 96.  96.  96.  96.  96.  96.  96.  96.]


In [36]:
np.corrcoef(Xp, rowvar=0)

array([[ 1.        ,  0.28052138,  0.22499988,  0.0273497 ,  0.538845  ,
         0.67531048,  0.43241706,  0.43365225],
       [ 0.28052138,  1.        ,  0.34796911,  0.4422644 ,  0.1553849 ,
         0.16453714,  0.05688209,  0.10735379],
       [ 0.22499988,  0.34796911,  1.        ,  0.3501859 ,  0.11765804,
         0.12766775,  0.2688916 ,  0.27611245],
       [ 0.0273497 ,  0.4422644 ,  0.3501859 ,  1.        , -0.08584324,
        -0.00699943,  0.07782045,  0.07846002],
       [ 0.538845  ,  0.1553849 ,  0.11765804, -0.08584324,  1.        ,
         0.67311118,  0.32041222,  0.45764762],
       [ 0.67531048,  0.16453714,  0.12766775, -0.00699943,  0.67311118,
         1.        ,  0.51483006,  0.63152825],
       [ 0.43241706,  0.05688209,  0.2688916 ,  0.07782045,  0.32041222,
         0.51483006,  1.        ,  0.75190451],
       [ 0.43365225,  0.10735379,  0.27611245,  0.07846002,  0.45764762,
         0.63152825,  0.75190451,  1.        ]])

In [38]:
df_preds

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0
3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20
4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0
5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0
6,-1.049822,3.228826,50,-1.386294,0,-1.386294,6,0
7,0.737164,3.473518,64,0.615186,0,-1.386294,6,0
8,0.693147,3.539509,58,1.536867,0,-1.386294,6,0
9,-0.776529,3.539509,47,-1.386294,0,-1.386294,6,0
10,0.223144,3.244544,63,-1.386294,0,-1.386294,6,0


In [40]:
df_preds.loc[df['train']=='T'].corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.300232,0.286324,0.063168,0.592949,0.692043,0.426414,0.483161
lweight,0.300232,1.000000,0.316723,0.437042,0.181054,0.156829,0.023558,0.074166
age,0.286324,0.316723,1.000000,0.287346,0.128902,0.172951,0.365915,0.275806
lbph,0.063168,0.437042,0.287346,1.000000,-0.139147,-0.088535,0.032992,-0.030404
svi,0.592949,0.181054,0.128902,-0.139147,1.000000,0.671240,0.306875,0.481358
lcp,0.692043,0.156829,0.172951,-0.088535,0.671240,1.000000,0.476437,0.662533
gleason,0.426414,0.023558,0.365915,0.032992,0.306875,0.476437,1.000000,0.757056
pgg45,0.483161,0.074166,0.275806,-0.030404,0.481358,0.662533,0.757056,1.000000


In [134]:
from sklearn.linear_model import LinearRegression

In [149]:
# fit_intercept = False b/c we include the intercept in X 
mdl = LinearRegression(fit_intercept=False)
res = mdl.fit(X,y)
res.coef_

array([[ 2.45234509,  0.71640701,  0.2926424 , -0.14254963,  0.2120076 ,
         0.30961953, -0.28900562, -0.02091352,  0.27734595]])

In [151]:
# this is 0 b/c we had fit_intercept=False
res.intercept_

0.0